<a href="https://colab.research.google.com/github/Navaneetdatta/CE888-repositor/blob/main/Lab%208/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-04 20:53:40--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  --.-KB/s    in 0.07s   

2021-05-04 20:53:40 (259 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/cats/cat

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [ ]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_image_sizes = []
cats_image_sizes = []
dogs_image_sizes = []
horses_image_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_image_sizes.append(hu.shape[:-1])
  cats_image_sizes.append(c.shape[:-1])
  dogs_image_sizes.append(d.shape[:-1])
  horses_image_sizes.append(ho.shape[:-1])

In [ ]:
mean_sizes = [np.mean(human_image_sizes, axis=0), np.mean(cats_image_sizes, axis=0),
              np.mean(dogs_image_sizes, axis=0), np.mean(horses_image_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [ ]:
image_generator = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [ ]:
train_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

image_size = [304,329,3]

In [ ]:
input = Input(shape = image_size)
model = ResNet50(input_tensor=input,)

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")


In [ ]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]


In [ ]:
model.fit(train_data, validation_data=val_data, epochs=50, callbacks=my_callbacks)


Epoch 1/50
182/182 [==============================] - 56s 117ms/step - loss: 1.4802 - accuracy: 0.2596 - val_loss: 1.4951 - val_accuracy: 0.2500
Epoch 2/50
182/182 [==============================] - 21s 114ms/step - loss: 1.3657 - accuracy: 0.3070 - val_loss: 1.2885 - val_accuracy: 0.3375
Epoch 3/50
182/182 [==============================] - 21s 113ms/step - loss: 1.2970 - accuracy: 0.3403 - val_loss: 1.4120 - val_accuracy: 0.4000
Epoch 4/50
182/182 [==============================] - 20s 111ms/step - loss: 1.3414 - accuracy: 0.3760 - val_loss: 1.2466 - val_accuracy: 0.3250
Epoch 5/50
182/182 [==============================] - 20s 112ms/step - loss: 1.2909 - accuracy: 0.3853 - val_loss: 1.2711 - val_accuracy: 0.3500
Epoch 6/50
182/182 [==============================] - 20s 111ms/step - loss: 1.2682 - accuracy: 0.3885 - val_loss: 1.2558 - val_accuracy: 0.4500
Epoch 7/50
182/182 [==============================] - 20s 111ms/step - loss: 1.2088 - accuracy: 0.4068 - val_loss: 1.2445 - val_ac

In [ ]:
model.history.history


{'accuracy': [0.28434064984321594,
  0.3186813294887543,
  0.35164836049079895,
  0.39148351550102234,
  0.3887362778186798,
  0.41208791732788086,
  0.39148351550102234,
  0.46291208267211914,
  0.40109890699386597,
  0.4711538553237915,
  0.47252747416496277,
  0.4436813294887543,
  0.48763737082481384,
  0.4670329689979553,
  0.4615384638309479,
  0.47252747416496277,
  0.4931318759918213,
  0.531593382358551],
 'loss': [1.4346157312393188,
  1.370989203453064,
  1.299083948135376,
  1.3086735010147095,
  1.2706689834594727,
  1.252694010734558,
  1.2189615964889526,
  1.2097364664077759,
  1.2197071313858032,
  1.1683354377746582,
  1.1894692182540894,
  1.1872917413711548,
  1.1650089025497437,
  1.1878854036331177,
  1.1615616083145142,
  1.1525956392288208,
  1.1375503540039062,
  1.102325677871704],
 'val_accuracy': [0.25,
  0.3375000059604645,
  0.4000000059604645,
  0.32499998807907104,
  0.3499999940395355,
  0.44999998807907104,
  0.4000000059604645,
  0.5375000238418579,
 